In this notebook, we will calculate features using Trotterization with a noise-free classical simulator (AerSimulator).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['x', 'measure', 'reset', 'cz', 'id', 'sx']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('x', (0,)), ('x', (1,)), ('x', (2,))

### Prepare parameters and load dataset

In [7]:
n_qubits = 40

C = 3
n_samples = 55
n_shots = 1000

if n_qubits <= 20:  # n_qubits <= 20
    threshold = 0.6

    # Load dataset (orient="records")
    if n_qubits == 20:  # 20 量子ビットの場合は、12 量子ビットの結果を使う
        df_n_step = pd.read_json(
            f"results/get_nstep/12Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    else:
        df_n_step = pd.read_json(
            f"results/get_nstep/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 20
    threshold = None
    n_features = 12
    # 間隔 np.pi/C で等間隔な点を n_features 個生成
    times = np.linspace(0, np.pi / C * (n_features - 1), n_features)

    # n_qubits > 20 の場合は、n_step は 1 から max_n_step の整数値を取る
    if n_qubits == 32:
        max_n_step = 4
    elif n_qubits == 40:
        max_n_step = 5
    # まず 1 から max_n_step までの n_features 個の等間隔の数値を生成
    n_step_array = np.linspace(1, max_n_step, n_features)
    # 各値を四捨五入して整数に変換
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

n_qubits: 40
C: 3
threshold: None
n_samples: 55
n_shots: 1000
n_features: 12
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306]
n_step_array: [1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.027569343700000002, -0.0061509184, 0.036087...",0.913388,NaN,NaN
1,1,"[0.0198111334, -0.0073320913, 0.0388114779, 0....",1.057062,NaN,NaN
2,2,"[0.0059292635, 0.0295150582, 0.017082614700000...",1.009956,NaN,NaN
3,3,"[0.0255905961, -0.0079183276, 0.0148449159, 0....",1.115899,NaN,NaN
4,4,"[-0.0199742169, 0.0145559143, 0.0207255021, 0....",0.906205,NaN,NaN
5,5,"[-0.0313871088, 0.009725694600000001, 0.036654...",0.956448,NaN,NaN
6,6,"[0.0397126215, 0.0279129831, -0.00299143800000...",1.192622,NaN,NaN
7,7,"[0.0097647419, 0.0193609483, -0.0113959584, -0...",0.961146,NaN,NaN
8,8,"[-0.0277423153, -0.013743889300000001, -0.0138...",0.915409,NaN,NaN
9,9,"[-0.035715704, 0.0070571812, 0.0078699068, -0....",0.926034,NaN,NaN


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [8]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    "backend": backend_mps_noiseless,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [11]:
# For simulator, we need job instances instead of job ids
# 進捗は results/fourier_feature_sim_noisy/temp_progress.txt に保存される
_, jobs, _ = run_job(config, backend_qpu, sim_type="noiseless")

In [12]:
%%time
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,f_11 Re,f_11 Im,expected_value
0,0,1.0,0.995068,-0.101603,0.985769,-0.152131,0.966971,-0.242307,0.917332,-0.367926,...,-0.579074,0.696426,-0.656533,0.625493,-0.703141,0.527739,-0.760180,0.448999,-0.779657,0.913388
1,1,1.0,0.993863,0.028282,0.966114,0.121276,0.927225,0.124671,0.874305,0.202482,...,0.270189,0.523557,0.310044,0.397968,0.306722,0.258020,0.328185,0.156160,0.307816,1.057062
2,2,1.0,0.998570,0.025007,1.001678,-0.005030,0.984344,0.042665,0.983537,0.040251,...,0.083842,0.916035,0.062172,0.877699,0.136531,0.880402,0.135828,0.825396,0.164067,1.009956
3,3,1.0,0.997161,0.071489,0.951670,0.252586,0.946109,0.295162,0.874687,0.412879,...,0.661237,0.534475,0.727188,0.444772,0.760147,0.328928,0.788315,0.219938,0.764139,1.115899
4,4,1.0,0.997019,-0.082075,0.962747,-0.267810,0.942595,-0.306352,0.902052,-0.396488,...,-0.657430,0.654550,-0.692865,0.564455,-0.759968,0.459197,-0.821086,0.347311,-0.836278,0.906205
5,5,1.0,0.996926,-0.063426,0.969140,-0.077408,0.915941,-0.117781,0.889453,-0.176130,...,-0.222388,0.685869,-0.198117,0.610900,-0.205245,0.578321,-0.157805,0.544963,-0.112336,0.956448
6,6,1.0,0.986924,0.154685,0.927557,0.341034,0.863408,0.488064,0.739228,0.633235,...,0.852364,0.115447,0.856780,-0.007720,0.844157,-0.183690,0.771432,-0.299204,0.693220,1.192622
7,7,1.0,0.997699,-0.085174,0.989118,-0.063486,0.975450,-0.135077,0.972204,-0.150565,...,-0.235535,0.867140,-0.230055,0.832938,-0.224787,0.813951,-0.299117,0.786500,-0.331432,0.961146
8,8,1.0,0.993759,-0.092157,0.969903,-0.179013,0.927322,-0.273556,0.855182,-0.367035,...,-0.468502,0.501654,-0.479099,0.439143,-0.492692,0.347854,-0.500407,0.260524,-0.410965,0.915409
9,9,1.0,0.994881,-0.082579,0.962395,-0.178777,0.921911,-0.241967,0.844978,-0.313747,...,-0.436148,0.613728,-0.417573,0.523106,-0.447681,0.422031,-0.384856,0.344970,-0.366439,0.926034


CPU times: user 6h 59min 18s, sys: 2h 5min 45s, total: 9h 5min 3s
Wall time: 13min 17s
